# Pseudogroup-labels by splitting training set
The idea is that naive ERM model will have an inherit bias given certain training set and one way to figure out such a bias is through using a validation set (i.e. the initial model would fail on these validation points while having 100% accuracy on the training set). 
Hence, the idea is to partition the training data into two parts: first part is used to train the the initial model (with fraction f of the training data) while the second part is used to "retrain" the model (either fine-tuning the last layer or fine-tuning the entire model or retraining the entire model). 
Doing so will enable:
1. If the initial features are good, then finetuning the last model (i.e. the linear classifier on top of these features) using the pseudogroup-labelled second dataset will hopefully give us a much better classifier rather than the initial biased classifier that might use certain spurious features. There is evidence for this behavior in the long-tail literature (e.g. tau-norm)
2. If the features are not that good, then retraining the entire model (either with fresh initialization or on top of the previous intialization) using methods that incorporate these pseudolabels will hopefully help the model obtain results close to when we have the true labels. 
    - Retraining the entire model might require a much higher sample complexity than just finetuning the last layer and there's an obvious tradeoff between these methods here. 

The goal here is to reduce the naive ERM model's spurious correlation. There might be further spurious correlations but the assumption is that the naive ERM model's spurious correlation might be the most obvious one that datasets in this worst-group-error area tends to focus on. To solve this in general would be to perfectly model the data distribution, and that seems much more difficult (domain generalization). 

## Method
This method, unlike many existing methods in the literature, aims to improve worst-group error by generating pseudogroup labels without real group labels (so that we can perform model selection... also we can have a long discussion on what "real group labels" mean) while minimizing the amount of hyperparameters required for proper implementation. 
We perform the following steps:
1. Partition the training set into part1 and part2 with $f$ and $(1-f)$ proportions, respectively.
2. Train an initial ERM model $I$ to convergence (either 100% accuracy or doesn't improve) on part1 of the training set.
3. Evaluate part2 of the train set on $I$. For each example $(x,y) \in$ part2, record $(x,y,s)$ where $s=1[I(x)=y]$ is the status of whether $I$ correctly classifies $x$ or not. 
    - Generate pseudogroup labels for examples in part2: for every $(x,y)$ in part2, the group label is $(y,s)$. This implicitly implies that the algorithm is partitioning every label into two groups. This is the case with popular "pseudogroup-generating" algorithms like JTT, GEORGE, EIIL, etc.
4. Retrain model $I$ to $I'$ using the pseudogroup labels by either fine-tuning the last layer, fine-tuning the model $I$, or train a new model entirely.

# Implementation
## 1. Partitioning the train set into part1 and part2

In [1]:
%cd ~/research/pseudogroups/split_pgl
%load_ext autoreload
%autoreload 2

/home/thiennguyen/research/pseudogroups/split_pgl


In [2]:
from data.data import dataset_attributes, shift_types, prepare_data, log_data
from utils import set_seed, Logger, CSVBatchLogger, log_args, get_model, hinge_loss
from notebooks.example_args import MyCUBArgs
import torch 
from data import dro_dataset
import wandb, os
from train import train
from data.folds import Subset, ConcatDataset

mainargs = MyCUBArgs(wandb=False,
                     seed=0,
                     show_progress=False,
                     project_name='')  # default gpu = 0
part1_args = mainargs.part1_args
part2_args = mainargs.part2_args
args = part1_args
set_seed(args.seed)

def split_data(dataset, part1_proportion=0.5, seed=None):
    """
        Split data into 2 parts given a ratio.
    """
    random = np.random.RandomState(seed)
    n_exs = len(dataset)
    n_part1 = int(n_exs*part1_proportion)
    # n_part2 = n_exs - n_part1
    if type(dataset) == Subset:  # note that this Subset is data.folds.Subset
        idxs = np.random.permutation(dataset.indices)
        data = dataset.dataset
    else:  # this is not a Subset i.e. just an ordinary dataset
        idxs = np.random.permutation(np.arange(len(dataset)))
        data = dataset
    part1, part2 = Subset(data, idxs[:n_part1]), Subset(data, idxs[n_part1:])  # this is not torch subset
    return part1, part2

In [7]:
## Function given dataset and fraction f (is this implemented already?)
# first load the train set
train_data, val_data, test_data = prepare_data(args, train=True)
# then split it into part1 containing f*n examples of trainset and part2 containing the rest
part1, part2 = split_data(train_data.dataset, part1_proportion=0.5, seed=args.seed)

part1_data = dro_dataset.DRODataset(
    part1,
    process_item_fn=None,
    n_groups=train_data.n_groups,
    n_classes=train_data.n_classes,
    group_str_fn=train_data.group_str)

part2_data = dro_dataset.DRODataset(
    part2,
    process_item_fn=None,
    n_groups=train_data.n_groups,
    n_classes=train_data.n_classes,
    group_str_fn=train_data.group_str)

loader_kwargs = {
        "batch_size": args.batch_size,
        "num_workers": 4,
        "pin_memory": True,
    }
part1_loader = dro_dataset.get_loader(part1_data,
                                      train=True,
                                      reweight_groups=None,
                                      **loader_kwargs)

Reading './cub/data/waterbird_complete95_forest2water2/metadata.csv'


In [19]:
from utils import get_subsampled_indices
indices = get_subsampled_indices(part2_data)
for i in indices:
    print(part2_data[i][0].shape)

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([

In [23]:
import os

mydir = '/root1/second/third'
test = os.path.basename(os.path.dirname(mydir))
print(test)

second


## 2. Training the initial ERM model on part1

In [5]:
"""this would be in run_expt"""
RUN_TEST = False
mode = 'w'
args.wandb = False
if args.wandb:
    run = wandb.init(project=f"{args.project_name}_{args.dataset}")
    wandb.config.update(args)
    
## Initialize logs
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

logger = Logger(os.path.join(args.log_dir, "log.txt"), 'w')
# Record args
log_args(args, logger)
# set_seed(args.seed)

# get loader and run train ERM
loader_kwargs = {
        "batch_size": args.batch_size,
        "num_workers": 4,
        "pin_memory": True,
    }
part1_loader = dro_dataset.get_loader(part1_data,
                                      train=True,
                                      reweight_groups=None,
                                      **loader_kwargs)
val_loader = dro_dataset.get_loader(val_data,
                                    train=False,
                                    reweight_groups=None,
                                    **loader_kwargs)

data = {}
data["train_data"] = part1_data
data["train_loader"] = part1_loader  # we train using  part1 
data["val_data"] = val_data
data["val_loader"] = val_loader

data["test_data"] = test_data
data["test_loader"] = None

train_csv_logger = CSVBatchLogger(os.path.join(args.log_dir, f"train.csv"),
                                  train_data.n_groups,
                                  mode=mode)
val_csv_logger = CSVBatchLogger(os.path.join(args.log_dir, f"val.csv"),
                                val_data.n_groups,
                                mode=mode)
test_csv_logger = None

if RUN_TEST:
    test_loader = dro_dataset.get_loader(test_data,
                                     train=False,
                                     reweight_groups=None,
                                     **loader_kwargs)
    data["test_loader"] = test_loader
    test_csv_logger = CSVBatchLogger(os.path.join(args.log_dir, f"test.csv"),
                                 test_data.n_groups,
                                 mode=mode)

n_classes = train_data.n_classes

log_data(data, logger)
logger.flush()

criterion = torch.nn.CrossEntropyLoss(reduction="none")


PermissionError: [Errno 13] Permission denied: '/SPGL_proportion0.5_epochs51_lr0.001_weightdecay0.0001'

In [63]:
### INITIALIZE MODEL AND TRAIN ###
resume = False  # not resuming yet. 
## Initialize model
model = get_model(
    model=args.model,
    pretrained=not args.train_from_scratch,
    resume=resume,
    n_classes=train_data.n_classes,
    dataset=args.dataset,
    log_dir=args.log_dir,
)
if args.wandb:
    wandb.watch(model)

epoch_offset = 0

train(
    model,
    criterion,
    data,
    logger,
    train_csv_logger,
    val_csv_logger,
    test_csv_logger,
    args,
    epoch_offset=epoch_offset,
    csv_name=args.fold,
    wandb=wandb if args.wandb else None,
)

train_csv_logger.close()
val_csv_logger.close()
if RUN_TEST:
    test_csv_logger.close()


Epoch [0]:
Training:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_0.csv
logged to wandb
Average incurred loss: 0.318  
Average sample loss: 0.316  
Average acc: 0.875  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.170  exp loss = 0.111  adjusted loss = 0.111  adv prob = 0.250000   acc = 0.993
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.558  exp loss = 0.785  adjusted loss = 0.785  adv prob = 0.250000   acc = 0.705
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 1.396  exp loss = 1.349  adjusted loss = 1.349  adv prob = 0.250000   acc = 0.107
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.702  exp loss = 0.359  adjusted loss = 0.359  adv prob = 0.250000   acc = 0.560

Validation:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_val_epoch_0.csv
logged to wandb
Ave

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_test_epoch_4.csv
logged to wandb
Current lr: 0.001000


Epoch [5]:
Training:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_5.csv
logged to wandb
Average incurred loss: 0.023  
Average sample loss: 0.023  
Average acc: 0.997  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.009  exp loss = 0.008  adjusted loss = 0.008  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.074  exp loss = 0.061  adjusted loss = 0.061  adv prob = 0.250000   acc = 0.989
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.266  exp loss = 0.258  adjusted loss = 0.258  adv prob = 0.250000   acc = 0.893
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.046  exp loss = 0.041  adjusted loss = 0.041  adv prob = 0.250000   acc =


Validation:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_val_epoch_9.csv
logged to wandb
Average incurred loss: 0.544  
Average sample loss: 0.540  
Average acc: 0.817  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 467]:	loss = 0.016  exp loss = 0.014  adjusted loss = 0.014  adv prob = 0.250000   acc = 0.994
  waterbird_complete95 = 0, forest2water2 = 1  [n = 466]:	loss = 0.838  exp loss = 0.837  adjusted loss = 0.837  adv prob = 0.250000   acc = 0.695
  waterbird_complete95 = 1, forest2water2 = 0  [n = 133]:	loss = 1.773  exp loss = 1.930  adjusted loss = 1.930  adv prob = 0.250000   acc = 0.481
  waterbird_complete95 = 1, forest2water2 = 1  [n = 133]:	loss = 0.141  exp loss = 0.122  adjusted loss = 0.122  adv prob = 0.250000   acc = 0.962
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_test_epoch_9.csv
logged to wandb
Current lr: 0.001000


Ep

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_14.csv
logged to wandb
Average incurred loss: 0.003  
Average sample loss: 0.003  
Average acc: 1.000  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.002  exp loss = 0.002  adjusted loss = 0.002  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.009  exp loss = 0.015  adjusted loss = 0.015  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.030  exp loss = 0.036  adjusted loss = 0.036  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.006  exp loss = 0.004  adjusted loss = 0.004  adv prob = 0.250000   acc = 1.000

Validation:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_val_epoch_14.csv
logged to wandb
Average incurred loss: 

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_test_epoch_18.csv
logged to wandb
Current lr: 0.001000


Epoch [19]:
Training:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_19.csv
logged to wandb
Average incurred loss: 0.003  
Average sample loss: 0.003  
Average acc: 1.000  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.002  exp loss = 0.002  adjusted loss = 0.002  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.009  exp loss = 0.009  adjusted loss = 0.009  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.017  exp loss = 0.013  adjusted loss = 0.013  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.003  exp loss = 0.003  adjusted loss = 0.003  adv prob = 0.250000   ac

...

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_28.csv
logged to wandb
Average incurred loss: 0.001  
Average sample loss: 0.001  
Average acc: 1.000  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.001  exp loss = 0.001  adjusted loss = 0.001  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.003  exp loss = 0.002  adjusted loss = 0.002  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.007  exp loss = 0.007  adjusted loss = 0.007  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.003  exp loss = 0.002  adjusted loss = 0.002  adv prob = 0.250000   acc = 1.000

Validation:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_val_epoch_28.csv
logged to wandb
Average incurred loss: 

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_test_epoch_32.csv
logged to wandb
Current lr: 0.001000


Epoch [33]:
Training:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_33.csv
logged to wandb
Average incurred loss: 0.001  
Average sample loss: 0.001  
Average acc: 1.000  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.001  exp loss = 0.000  adjusted loss = 0.000  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.002  exp loss = 0.005  adjusted loss = 0.005  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.009  exp loss = 0.011  adjusted loss = 0.011  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.002  exp loss = 0.002  adjusted loss = 0.002  adv prob = 0.250000   ac


Validation:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_val_epoch_37.csv
logged to wandb
Average incurred loss: 0.685  
Average sample loss: 0.680  
Average acc: 0.810  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 467]:	loss = 0.018  exp loss = 0.015  adjusted loss = 0.015  adv prob = 0.250000   acc = 0.991
  waterbird_complete95 = 0, forest2water2 = 1  [n = 466]:	loss = 1.123  exp loss = 1.116  adjusted loss = 1.116  adv prob = 0.250000   acc = 0.665
  waterbird_complete95 = 1, forest2water2 = 0  [n = 133]:	loss = 2.021  exp loss = 2.145  adjusted loss = 2.145  adv prob = 0.250000   acc = 0.519
  waterbird_complete95 = 1, forest2water2 = 1  [n = 133]:	loss = 0.152  exp loss = 0.124  adjusted loss = 0.124  adv prob = 0.250000   acc = 0.970
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_test_epoch_37.csv
logged to wandb
Current lr: 0.001000




Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_42.csv
logged to wandb
Average incurred loss: 0.001  
Average sample loss: 0.001  
Average acc: 1.000  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.000  exp loss = 0.000  adjusted loss = 0.000  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.001  exp loss = 0.001  adjusted loss = 0.001  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.003  exp loss = 0.003  adjusted loss = 0.003  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.001  exp loss = 0.001  adjusted loss = 0.001  adv prob = 0.250000   acc = 1.000

Validation:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_val_epoch_42.csv
logged to wandb
Average incurred loss: 

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_test_epoch_46.csv
logged to wandb
Current lr: 0.001000


Epoch [47]:
Training:
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/model_outputs/output_train_epoch_47.csv
logged to wandb
Average incurred loss: 0.001  
Average sample loss: 0.001  
Average acc: 1.000  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1745]:	loss = 0.000  exp loss = 0.000  adjusted loss = 0.000  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 0, forest2water2 = 1  [n = 88]:	loss = 0.001  exp loss = 0.001  adjusted loss = 0.001  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 0  [n = 28]:	loss = 0.007  exp loss = 0.007  adjusted loss = 0.007  adv prob = 0.250000   acc = 1.000
  waterbird_complete95 = 1, forest2water2 = 1  [n = 536]:	loss = 0.001  exp loss = 0.001  adjusted loss = 0.001  adv prob = 0.250000   ac

## 3. Evaluating part2 on the initial ERM model and generate pseudogroup labels for part2

### First evaluate part2 on initial model

In [2]:
!ls /home/thiennguyen/research/pseudogroups/CUB/autolog_splitpgl/erm_upweight-1_epochs101_lr0.001_wd0.0001/part1

0_model.pth		    output_train_epoch_64.csv  output_val_epoch_39.csv
100_model.pth		    output_train_epoch_65.csv  output_val_epoch_3.csv
10_model.pth		    output_train_epoch_66.csv  output_val_epoch_40.csv
20_model.pth		    output_train_epoch_67.csv  output_val_epoch_41.csv
30_model.pth		    output_train_epoch_68.csv  output_val_epoch_42.csv
40_model.pth		    output_train_epoch_69.csv  output_val_epoch_43.csv
50_model.pth		    output_train_epoch_6.csv   output_val_epoch_44.csv
60_model.pth		    output_train_epoch_70.csv  output_val_epoch_45.csv
70_model.pth		    output_train_epoch_71.csv  output_val_epoch_46.csv
80_model.pth		    output_train_epoch_72.csv  output_val_epoch_47.csv
90_model.pth		    output_train_epoch_73.csv  output_val_epoch_48.csv
log.txt			    output_train_epoch_74.csv  output_val_epoch_49.csv
output_train_epoch_0.csv    output_train_epoch_75.csv  output_val_epoch_4.csv
output_train_epoch_100.csv  output_train_epoch_76.csv  output_val_epoch_50.csv
output_train_epoch_1

In [12]:
from train import run_epoch
from loss import LossComputer
import numpy as np 

part2_log_dir = "/home/thiennguyen/research/pseudogroups/CUB/autolog_splitpgl/erm_upweight-1_epochs101_lr0.001_wd0.0001/part2_eval"
if not os.path.exists(part2_log_dir):
    os.makedirs(part2_log_dir)
part2_logger = Logger(os.path.join(part2_log_dir, "log.txt"), 'w')
part2_logger.flush()

part1_log_dir = "/home/thiennguyen/research/pseudogroups/CUB/autolog_splitpgl/erm_upweight-1_epochs101_lr0.001_wd0.0001/part1/"
model_path = f"{part1_log_dir}/90_model.pth"
# first load the previous model
modeleval = torch.load(model_path)

In [18]:
# initialize logger and loader for part2
mode='w'
train_data, val_data, test_data = prepare_data(args, train=True)
criterion = torch.nn.CrossEntropyLoss(reduction="none")
loader_kwargs = {
        "batch_size": args.batch_size,
        "num_workers": 4,
        "pin_memory": True}
part2_data = torch.load(os.path.join(part1_log_dir,"part1and2_data"))["part2"]
part2eval_csv_logger = CSVBatchLogger(os.path.join(part2_log_dir, f"part2_eval.csv"),
                                 test_data.n_groups,
                                 mode=mode)
part2_loader = dro_dataset.get_loader(part2_data,
                                      train=False,
                                      reweight_groups=None,
                                      **loader_kwargs)
adjustments = np.array([float(c) for c in args.generalization_adjustment.split(",")]* 4)
part2_loss_computer = LossComputer(
            criterion,
            loss_type=args.loss_type,
            dataset=train_data,
            alpha=args.alpha,
            gamma=args.gamma,
            adj=adjustments,
            step_size=args.robust_step_size,
            normalize_loss=args.use_normalized_loss,
            btl=args.btl,
            min_var_weight=args.minimum_variational_weight,
            joint_dro_alpha=args.joint_dro_alpha,
        )

# then run an epoch on part2 and during that run, generate a csv containing the status of each example 
run_epoch(
    epoch=0,
    model=modeleval,
    optimizer=None,
    loader=part2_loader,
    loss_computer=part2_loss_computer,
    logger=part2_logger,
    csv_logger=part2eval_csv_logger,
    args=args,
    is_training=False,
    show_progress=True,
    log_every=50,
    scheduler=None,
    csv_name="pseudogroup_eval",
    wandb_group="part2_eval",
    wandb=wandb,
)

Reading './cub/data/waterbird_complete95_forest2water2/metadata.csv'


100%|██████████████████████████| 38/38 [00:03<00:00, 10.96it/s]

Saved /home/thiennguyen/research/pseudogroups/CUB/autolog_splitpgl/erm_upweight-1_epochs101_lr0.001_wd0.0001/part2_eval/output_part2_eval_epoch_0.csv
logged to wandb
Average incurred loss: 0.104  
Average sample loss: 0.105  
Average acc: 0.970  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1756]:	loss = 0.017  exp loss = 0.018  adjusted loss = 0.018  adv prob = 0.250000   acc = 0.995
  waterbird_complete95 = 0, forest2water2 = 1  [n = 91]:	loss = 1.219  exp loss = 1.541  adjusted loss = 1.541  adv prob = 0.250000   acc = 0.626
  waterbird_complete95 = 1, forest2water2 = 0  [n = 25]:	loss = 1.696  exp loss = 1.125  adjusted loss = 1.125  adv prob = 0.250000   acc = 0.720
  waterbird_complete95 = 1, forest2water2 = 1  [n = 526]:	loss = 0.127  exp loss = 0.091  adjusted loss = 0.091  adv prob = 0.250000   acc = 0.958


In [19]:
wandb.finish()

batch_idx,▁
epoch,▁
part2_eval/avg_acc,▁
part2_eval/avg_acc_group:0,▁
part2_eval/avg_acc_group:1,▁
part2_eval/avg_acc_group:2,▁
part2_eval/avg_acc_group:3,▁
part2_eval/avg_actual_loss,▁
part2_eval/avg_loss_group:0,▁
part2_eval/avg_loss_group:1,▁
part2_eval/avg_loss_group:2,▁


### Then generate pseudogroup labels

In [74]:
import pandas as pd
from data.folds import ConcatDataset

# want to create a sampler so that misclassified examples of each label is sampled "more frequently"
# the pseudogroup labels are located in part2_log_dir/output_eval_epoch_0.csv under 
#     y_pred_pseudogroup_eval_epoch_0_val and y_true_pseudogroup_eval_epoch_0_val columns...
#     the group is (y_true, y_pred==y_true)
part2_df = pd.read_csv(os.path.join(part2_log_dir,'output_part2_eval_epoch_0.csv'))
misclassified = part2_df['y_pred_pseudogroup_eval_epoch_0_val'] != part2_df['y_true_pseudogroup_eval_epoch_0_val']
aug_indices = part2_df['indices_pseudogroup_eval_epoch_0_val'][misclassified]
upweight_factor = len(part2_df)//len(aug_indices)
combined_indices = list(aug_indices) * upweight_factor + list(part2_df['indices_pseudogroup_eval_epoch_0_val'])
upsampled_part2 = torch.utils.data.Subset(train_data.dataset.dataset, combined_indices)

In [27]:
import pandas as pd
from data.folds import ConcatDataset

n_classes = part2_data.n_classes
part2_df = pd.read_csv(os.path.join(part2_log_dir, 'output_part2_eval_epoch_0.csv'))
n_groups = n_classes*2
true_y = part2_df['y_true_pseudogroup_eval_epoch_0_val']
pred_y = part2_df['y_pred_pseudogroup_eval_epoch_0_val']
misclassified = true_y != pred_y
group_array = misclassified + true_y*2  # times 2 to make group (true_y, status)
# print(group_array)
group_counts = (torch.arange(n_groups).unsqueeze(1) == torch.tensor(group_array)).sum(1).float()
print(group_counts)
group_weights = len(part2_data)/ group_counts
print(group_weights)
weights = group_weights[group_array]

tensor([1805.,   42.,  522.,   29.])
tensor([ 1.3285, 57.0952,  4.5939, 82.6897])


In [75]:
# part2_df['indices_pseudogroup_eval_epoch_0_val'][idx] == part2_data.dataset.indices[idx]
# we can use this match up to create our weighted sampler

0 1365 [0]
1 11701 [1]
2 1249 [2]
3 9201 [3]
4 6565 [4]
5 4241 [5]
6 445 [6]
7 8111 [7]
8 7527 [8]
9 4984 [9]
10 627 [10]
11 10578 [11]
12 9282 [12]
13 1922 [13]
14 6293 [14]
15 3901 [15]
16 644 [16]
17 9843 [17]
18 7644 [18]
19 3404 [19]
20 872 [20]
21 4839 [21]
22 2002 [22]
23 8960 [23]
24 4007 [24]
25 11075 [25]
26 2470 [26]
27 4608 [27]
28 4347 [28]
29 5278 [29]
30 1600 [30]
31 8336 [31]
32 654 [32]
33 1820 [33]
34 11138 [34]
35 11217 [35]
36 1581 [36]
37 7287 [37]
38 599 [38]
39 9069 [39]
40 7978 [40]
41 600 [41]
42 8963 [42]
43 8192 [43]
44 7163 [44]
45 2242 [45]
46 1683 [46]
47 7413 [47]
48 4448 [48]
49 9128 [49]
50 220 [50]
51 10235 [51]
52 10279 [52]
53 5195 [53]
54 2105 [54]
55 5641 [55]
56 6266 [56]
57 4342 [57]
58 5618 [58]
59 4404 [59]
60 1301 [60]
61 994 [61]
62 9935 [62]
63 363 [63]
64 7799 [64]
65 2319 [65]
66 4490 [66]
67 5415 [67]
68 8229 [68]
69 1367 [69]
70 9294 [70]
71 5075 [71]
72 8339 [72]
73 2464 [73]
74 8082 [74]
75 9268 [75]
76 5321 [76]
77 9312 [77]
78 2252 [

1069 3994 [1069]
1070 595 [1070]
1071 9260 [1071]
1072 11299 [1072]
1073 2849 [1073]
1074 9484 [1074]
1075 9316 [1075]
1076 9571 [1076]
1077 899 [1077]
1078 11237 [1078]
1079 402 [1079]
1080 11633 [1080]
1081 8497 [1081]
1082 8211 [1082]
1083 2520 [1083]
1084 7186 [1084]
1085 3262 [1085]
1086 2534 [1086]
1087 8701 [1087]
1088 5683 [1088]
1089 9490 [1089]
1090 8812 [1090]
1091 4112 [1091]
1092 1504 [1092]
1093 4436 [1093]
1094 2691 [1094]
1095 3091 [1095]
1096 5636 [1096]
1097 5361 [1097]
1098 7015 [1098]
1099 3507 [1099]
1100 5703 [1100]
1101 2383 [1101]
1102 7483 [1102]
1103 10699 [1103]
1104 526 [1104]
1105 8842 [1105]
1106 120 [1106]
1107 7156 [1107]
1108 8575 [1108]
1109 11301 [1109]
1110 4522 [1110]
1111 504 [1111]
1112 6561 [1112]
1113 1018 [1113]
1114 5129 [1114]
1115 5465 [1115]
1116 9064 [1116]
1117 2216 [1117]
1118 5517 [1118]
1119 3113 [1119]
1120 11749 [1120]
1121 9800 [1121]
1122 4111 [1122]
1123 7269 [1123]
1124 10820 [1124]
1125 8883 [1125]
1126 19 [1126]
1127 4409 [1127

2145 6107 [2145]
2146 5795 [2146]
2147 2747 [2147]
2148 1014 [2148]
2149 11220 [2149]
2150 5095 [2150]
2151 6470 [2151]
2152 1897 [2152]
2153 8868 [2153]
2154 3362 [2154]
2155 11243 [2155]
2156 5466 [2156]
2157 2325 [2157]
2158 969 [2158]
2159 6895 [2159]
2160 4289 [2160]
2161 490 [2161]
2162 10323 [2162]
2163 7038 [2163]
2164 4135 [2164]
2165 5236 [2165]
2166 422 [2166]
2167 4285 [2167]
2168 6924 [2168]
2169 10264 [2169]
2170 1119 [2170]
2171 1573 [2171]
2172 7100 [2172]
2173 10816 [2173]
2174 6023 [2174]
2175 7701 [2175]
2176 3786 [2176]
2177 4768 [2177]
2178 11005 [2178]
2179 4382 [2179]
2180 8354 [2180]
2181 5911 [2181]
2182 1488 [2182]
2183 2810 [2183]
2184 11344 [2184]
2185 10122 [2185]
2186 4848 [2186]
2187 7542 [2187]
2188 11720 [2188]
2189 10417 [2189]
2190 2687 [2190]
2191 10026 [2191]
2192 6282 [2192]
2193 5901 [2193]
2194 1352 [2194]
2195 6757 [2195]
2196 1580 [2196]
2197 496 [2197]
2198 4541 [2198]
2199 4880 [2199]
2200 7473 [2200]
2201 4379 [2201]
2202 1003 [2202]
2203 30

In [55]:
print(part2_data.dataset.indices)

[ 1365 11701  1249 ...  4940 10277 10248]


## 4. Train new model using pseudogroup labels 

In [ ]:
args = args_retrain_CUB

RETRAIN=False  # set this to False to use old model... else we are using new model
ONLY_LAST_LAYER=args.only_last_layer  # this this to True to only train the last layer.

"""this would be in run_expt"""
args.wandb = True  # don't set this yet...
mode = 'w'
if args.wandb:
    run = wandb.init(project=f"{args.project_name}_{args.dataset}")
    wandb.config.update(args)
    
## Initialize logs
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

logger = Logger(os.path.join(args.log_dir, "log.txt"), 'w')
# Record args
log_args(args, logger)

set_seed(args.seed)

# get loader and run train ERM
loader_kwargs = {
        "batch_size": args.batch_size,
        "num_workers": 4,
        "pin_memory": True,
    }
part2_rw_loader = dro_dataset.get_loader(upsampled_part2,
                                      train=True,
                                      reweight_groups=None,
                                      **loader_kwargs)

data = {}
data["train_loader"] = part2_rw_loader  # we train using part2 rw now!
data["val_loader"] = val_loader
data["test_loader"] = test_loader
data["train_data"] = part2_data  
data["val_data"] = val_data
data["test_data"] = test_data

n_classes = train_data.n_classes

log_data(data, logger)
logger.flush()

criterion = torch.nn.CrossEntropyLoss(reduction="none")
### INITIALIZE MODEL AND TRAIN ###
resume = False  # not resuming yet. 
## Initialize model
if RETRAIN:
    model = get_model(
        model=args.model,
        pretrained=not args.train_from_scratch,
        resume=resume,
        n_classes=train_data.n_classes,
        dataset=args.dataset,
        log_dir=args.log_dir,
    )
else:
    model = torch.load(model_path)  # this model_path is from the cell above -- fix this in code
    
if ONLY_LAST_LAYER:
    assert not RETRAIN, "is this intentional? Retraining and only training last layer --> linear classifier on random features?"
    # freeze everything except the last layer
    for name, param in model.named_parameters():
        if name not in ['fc.weight', 'fc.bias']:
            param.requires_grad = False
    # make sure freezing really worked
    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    assert len(parameters) == 2  # fc.weight, fc.bias
    
if args.wandb:
    wandb.watch(model)

epoch_offset = 0

train_csv_logger = CSVBatchLogger(os.path.join(args.log_dir, f"train.csv"),
                                  train_data.n_groups,
                                  mode=mode)
val_csv_logger = CSVBatchLogger(os.path.join(args.log_dir, f"val.csv"),
                                val_data.n_groups,
                                mode=mode)
test_csv_logger = CSVBatchLogger(os.path.join(args.log_dir, f"test.csv"),
                                 test_data.n_groups,
                                 mode=mode)

train(
    model,
    criterion,
    data,
    logger,
    train_csv_logger,
    val_csv_logger,
    test_csv_logger,
    args,
    epoch_offset=epoch_offset,
    csv_name=args.fold,
    wandb=wandb if args.wandb else None,
)

train_csv_logger.close()
val_csv_logger.close()
test_csv_logger.close()


Training Data...
    waterbird_complete95 = 0, forest2water2 = 0: n = 1753
    waterbird_complete95 = 0, forest2water2 = 1: n = 96
    waterbird_complete95 = 1, forest2water2 = 0: n = 28
    waterbird_complete95 = 1, forest2water2 = 1: n = 521
Validation Data...
    waterbird_complete95 = 0, forest2water2 = 0: n = 467
    waterbird_complete95 = 0, forest2water2 = 1: n = 466
    waterbird_complete95 = 1, forest2water2 = 0: n = 133
    waterbird_complete95 = 1, forest2water2 = 1: n = 133
Test Data...
    waterbird_complete95 = 0, forest2water2 = 0: n = 2255
    waterbird_complete95 = 0, forest2water2 = 1: n = 2255
    waterbird_complete95 = 1, forest2water2 = 0: n = 642
    waterbird_complete95 = 1, forest2water2 = 1: n = 642
Training for 51 epochs...

Epoch [0]:
Training:


 66%|█▉ | 49/74 [00:04<00:02, 12.41it/s]

Average incurred loss: 0.945  
Average sample loss: 0.945  
Average acc: 0.687  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1334]:	loss = 0.150  exp loss = 0.187  adjusted loss = 0.187  adv prob = 0.250000   acc = 0.945
  waterbird_complete95 = 0, forest2water2 = 1  [n = 828]:	loss = 1.270  exp loss = 0.821  adjusted loss = 0.821  adv prob = 0.250000   acc = 0.536
  waterbird_complete95 = 1, forest2water2 = 0  [n = 322]:	loss = 2.220  exp loss = 1.209  adjusted loss = 1.209  adv prob = 0.250000   acc = 0.366
  waterbird_complete95 = 1, forest2water2 = 1  [n = 716]:	loss = 1.479  exp loss = 1.055  adjusted loss = 1.055  adv prob = 0.250000   acc = 0.525


100%|███| 74/74 [00:06<00:00, 11.51it/s]

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/retrain_part2_upweight10/output_train_epoch_0.csv
logged to wandb
Average incurred loss: 0.358  
Average sample loss: 0.358  
Average acc: 0.850  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 635]:	loss = 0.197  exp loss = 0.215  adjusted loss = 0.215  adv prob = 0.250000   acc = 0.946
  waterbird_complete95 = 0, forest2water2 = 1  [n = 375]:	loss = 0.311  exp loss = 0.300  adjusted loss = 0.300  adv prob = 0.250000   acc = 0.877
  waterbird_complete95 = 1, forest2water2 = 0  [n = 165]:	loss = 0.455  exp loss = 0.516  adjusted loss = 0.516  adv prob = 0.250000   acc = 0.818
  waterbird_complete95 = 1, forest2water2 = 1  [n = 345]:	loss = 0.661  exp loss = 0.658  adjusted loss = 0.658  adv prob = 0.250000   acc = 0.658

Validation:


...

Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/retrain_part2_upweight10/output_train_epoch_48.csv
logged to wandb
Average incurred loss: 0.053  
Average sample loss: 0.053  
Average acc: 0.993  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 625]:	loss = 0.039  exp loss = 0.036  adjusted loss = 0.036  adv prob = 0.250000   acc = 0.995
  waterbird_complete95 = 0, forest2water2 = 1  [n = 398]:	loss = 0.051  exp loss = 0.056  adjusted loss = 0.056  adv prob = 0.250000   acc = 0.992
  waterbird_complete95 = 1, forest2water2 = 0  [n = 149]:	loss = 0.074  exp loss = 0.063  adjusted loss = 0.063  adv prob = 0.250000   acc = 0.993
  waterbird_complete95 = 1, forest2water2 = 1  [n = 348]:	loss = 0.072  exp loss = 0.079  adjusted loss = 0.079  adv prob = 0.250000   acc = 0.991

Validation:


Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/retrain_part2_upweight10/output_val_epoch_48.csv
logged to wandb
Average incurred loss: 0.378  
Average sample loss: 0.379  
Average acc: 0.877  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 467]:	loss = 0.053  exp loss = 0.046  adjusted loss = 0.046  adv prob = 0.250000   acc = 0.972
  waterbird_complete95 = 0, forest2water2 = 1  [n = 466]:	loss = 0.476  exp loss = 0.458  adjusted loss = 0.458  adv prob = 0.250000   acc = 0.843
  waterbird_complete95 = 1, forest2water2 = 0  [n = 133]:	loss = 1.209  exp loss = 1.286  adjusted loss = 1.286  adv prob = 0.250000   acc = 0.669
  waterbird_complete95 = 1, forest2water2 = 1  [n = 133]:	loss = 0.350  exp loss = 0.323  adjusted loss = 0.323  adv prob = 0.250000   acc = 0.872
Saved myresults/CUB/waterbird_firstrun/ERM_upweight_0_epochs_51_lr_0.001_weight_decay_0.0001/retrain_part2_upweight10/output_test_epoch_48.csv
logged to wandb
Current lr: 0.

 66%|█▉ | 49/74 [00:04<00:02, 12.26it/s]

Average incurred loss: 0.061  
Average sample loss: 0.061  
Average acc: 0.988  
  waterbird_complete95 = 0, forest2water2 = 0  [n = 1353]:	loss = 0.039  exp loss = 0.022  adjusted loss = 0.022  adv prob = 0.250000   acc = 0.991
  waterbird_complete95 = 0, forest2water2 = 1  [n = 811]:	loss = 0.048  exp loss = 0.036  adjusted loss = 0.036  adv prob = 0.250000   acc = 0.994
  waterbird_complete95 = 1, forest2water2 = 0  [n = 319]:	loss = 0.075  exp loss = 0.083  adjusted loss = 0.083  adv prob = 0.250000   acc = 0.994
  waterbird_complete95 = 1, forest2water2 = 1  [n = 717]:	loss = 0.110  exp loss = 0.100  adjusted loss = 0.100  adv prob = 0.250000   acc = 0.974


 72%|██▏| 53/74 [00:04<00:01, 12.05it/s]